In [1]:
# The code was removed by Watson Studio for sharing.

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200620185441-0001
KERNEL_ID = 4c1799eb-71a4-494f-b483-b6421cdde3bd


[Row(userId='1', movieId='1', rating='4.0', timestamp='964982703'),
 Row(userId='1', movieId='3', rating='4.0', timestamp='964981247'),
 Row(userId='1', movieId='6', rating='4.0', timestamp='964982224'),
 Row(userId='1', movieId='47', rating='5.0', timestamp='964983815'),
 Row(userId='1', movieId='50', rating='5.0', timestamp='964982931')]

In [2]:

df_data_movies = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('movies.csv', 'moviereccomendation-donotdelete-pr-lcyd00n1rahpgl'))
df_data_movies.take(5)


[Row(movieId='1', title='Toy Story (1995)', genres='Adventure|Animation|Children|Comedy|Fantasy'),
 Row(movieId='2', title='Jumanji (1995)', genres='Adventure|Children|Fantasy'),
 Row(movieId='3', title='Grumpier Old Men (1995)', genres='Comedy|Romance'),
 Row(movieId='4', title='Waiting to Exhale (1995)', genres='Comedy|Drama|Romance'),
 Row(movieId='5', title='Father of the Bride Part II (1995)', genres='Comedy')]

In [3]:
# Create repartitionedRatings DataFrame across 10 partitions

repartitionedRatings = df_data_ratings.repartition(10)
print('Number of partitions for ratings DF: {}'.format(df_data_ratings.rdd.getNumPartitions()))

print('Number of partitions for the repartitionedRatings DF: {}'.format(repartitionedRatings.rdd.getNumPartitions()))



Number of partitions for ratings DF: 1
Number of partitions for the repartitionedRatings DF: 10


In [4]:
print('Number of ratings: {}'. format(repartitionedRatings.count()))

Number of ratings: 100836


In [5]:
# Load datasets in memory to improve performance (cache)

repartitionedRatings.cache()

DataFrame[userId: string, movieId: string, rating: string, timestamp: string]

In [6]:
print('Number of ratings: {}'.format(repartitionedRatings.count()))

Number of ratings: 100836


In [7]:
print('Number of ratings: {}'.format(repartitionedRatings.count()))

Number of ratings: 100836


In [8]:
# EXPLORE DATA WITH SPARK API
df_data_movies.show(truncate=False)

df_data_ratings.show(truncate=False)

+-------+-------------------------------------+-------------------------------------------+
|movieId|title                                |genres                                     |
+-------+-------------------------------------+-------------------------------------------+
|1      |Toy Story (1995)                     |Adventure|Animation|Children|Comedy|Fantasy|
|2      |Jumanji (1995)                       |Adventure|Children|Fantasy                 |
|3      |Grumpier Old Men (1995)              |Comedy|Romance                             |
|4      |Waiting to Exhale (1995)             |Comedy|Drama|Romance                       |
|5      |Father of the Bride Part II (1995)   |Comedy                                     |
|6      |Heat (1995)                          |Action|Crime|Thriller                      |
|7      |Sabrina (1995)                       |Comedy|Romance                             |
|8      |Tom and Huck (1995)                  |Adventure|Children               

In [9]:
# The code was removed by Watson Studio for sharing.

In [10]:
movies_file_name = 'movies.csv'
ratings_file_name = 'ratings.csv'

# inferScheme by default shows string for all columns. Override this by the following:
df_data_movies = spark.read.csv(project.get_file_url(movies_file_name), header=True, inferSchema=True).repartition(10).cache()
df_data_ratings = spark.read.csv(project.get_file_url(ratings_file_name), header=True, inferSchema=True).repartition(10).cache()


In [11]:
df_data_movies.printSchema()
df_data_ratings.printSchema()

root
 |-- movieId: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- genres: string (nullable = true)

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [12]:
df_data_ratings.describe().show()

+-------+------------------+----------------+------------------+--------------------+
|summary|            userId|         movieId|            rating|           timestamp|
+-------+------------------+----------------+------------------+--------------------+
|  count|            100836|          100836|            100836|              100836|
|   mean|326.12756356856676|19435.2957177992| 3.501556983616962|1.2059460873684695E9|
| stddev| 182.6184914634999|35530.9871987002|1.0425292390606344| 2.162610359951315E8|
|    min|                 1|               1|               0.5|           828124615|
|    max|               610|          193609|               5.0|          1537799250|
+-------+------------------+----------------+------------------+--------------------+



In [13]:
print('Number of different users: {}'.format(df_data_ratings.select('userId').distinct().count()))
print('Number of different movies: {}'.format(df_data_ratings.select('movieId').distinct().count()))
print('Number of movies with at least one rating strictly higher than 4: {}'.format(df_data_ratings.filter('rating > 4').select('movieId').distinct().count()))


Number of different users: 610
Number of different movies: 9724
Number of movies with at least one rating strictly higher than 4: 4056


In [14]:
df_data_ratings.createOrReplaceTempView('df_data_ratings')
spark.sql('Select COUNT(DISTINCT(movieId)) AS nb FROM df_data_ratings WHERE rating > 4').show()

+----+
|  nb|
+----+
|4056|
+----+



In [15]:
ratings_url = project.get_file_url(ratings_file_name)
sql = 'SELECT * FROM csv.`' + ratings_url + '`'
spark.sql(sql).take(2)


[Row(_c0='userId', _c1='movieId', _c2='rating', _c3='timestamp'),
 Row(_c0='1', _c1='1', _c2='4.0', _c3='964982703')]

In [16]:
!pip install --upgrade pandas

    100% |████████████████████████████████| 10.1MB 1.4MB/s eta 0:00:01
    100% |████████████████████████████████| 20.1MB 826kB/s eta 0:00:01
    100% |████████████████████████████████| 512kB 5.2MB/s eta 0:00:01
    100% |████████████████████████████████| 235kB 6.2MB/s eta 0:00:01
tensorflow 1.13.1 requires tensorboard<1.14.0,>=1.13.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is not installed.
watson-machine-learning-client-v4 1.0.95 has requirement pandas<=0.25.3, but you'll have pandas 1.0.5 which is incompatible.
pytest-openfiles 0.5.0 has requirement pytest>=4.6, but you'll have pytest 3.10.1 which is incompatible.
pytest-astropy 0.8.0 has requirement pytest>=4.6, but you'll have pytest 3.10.1 which is incompatible.


In [17]:
import pandas as pd
df_data_ratings.toPandas().head()

,userId,movieId,rating,timestamp
0,249,5803,3.0,1354225800
1,610,84772,3.5,1493846852
2,599,1982,3.0,1498524557
3,468,317,3.0,831400519
4,169,5873,4.5,1059429313


In [18]:
# VISUALIZE THE DATA
!pip install --upgrade seaborn

    100% |████████████████████████████████| 225kB 4.3MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/c0/95/cb9820560a2713384ef49060b0087dfa2591c6db6f240215c2bce1f4211c/pandas-1.0.5-cp36-cp36m-manylinux1_x86_64.whl
    100% |████████████████████████████████| 12.4MB 943kB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/b3/a9/b1bc4c935ed063766bce7d3e8c7b20bd52e515ff1c732b02caacf7918e5a/numpy-1.18.5-cp36-cp36m-manylinux1_x86_64.whl
    100% |████████████████████████████████| 26.1MB 738kB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/4f/a4/879454d49688e2fad93e59d7d4efda580b783c745fd2ec2a3adf87b0808d/pytz-2020.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl
    100% |████████████████████████████████| 71kB 2.9MB/s eta 0:00:01
    100% |████████████████████████████████| 92kB 5.2MB/s eta 0:

In [19]:
import seaborn as sns
import matplotlib.pyplot
%matplotlib inline



In [20]:
ratingsPandas = df_data_ratings.toPandas()
moviesPandas = df_data_movies.toPandas()

sns.lmplot(x='userId', y='movieId', data=ratingsPandas, fit_reg=False);

ImportError: libpng16-cfdb1654.so.16.21.0: cannot open shared object file: No such file or directory

<Figure size 360x360 with 1 Axes>

In [21]:
sns.palplot(sns.diverging_palette(10, 133, sep=80, n=10))


ImportError: libpng16-cfdb1654.so.16.21.0: cannot open shared object file: No such file or directory

<Figure size 720x72 with 1 Axes>

In [22]:
sns.violinplot([ratingsPandas.rating]);


ImportError: libpng16-cfdb1654.so.16.21.0: cannot open shared object file: No such file or directory

<Figure size 432x288 with 1 Axes>

In [23]:
# BUILD RECOMMENDER SYSTEM
spark.sql('''
    SELECT *, 100 * nb_ratings/matrix_size AS percentage
    FROM (
        SELECT nb_users, nb_movies, nb_ratings, nb_users * nb_movies AS matrix_size
        FROM (
            SELECT COUNT(*) AS nb_ratings, COUNT(DISTINCT(movieId)) AS nb_movies, COUNT(DISTINCT(userId)) AS nb_users
            FROM df_data_ratings
        )
    )
''').toPandas().head()


,nb_users,nb_movies,nb_ratings,matrix_size,percentage
0,610,9724,100836,5931640,1.699968


In [24]:
# Train the Model
# Use SparkML ALS Algorithm

from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

(trainingRatings, testRatings) = df_data_ratings.randomSplit([80.0, 20.0])
als = ALS(userCol='userId', itemCol='movieId', ratingCol='rating')
model = als.fit(trainingRatings)
trainingRatings.toPandas()

,userId,movieId,rating,timestamp
0,1,47,5.0,964983815
1,1,356,4.0,964980962
2,1,367,4.0,964981710
3,1,527,5.0,964984002
4,1,592,4.0,964982271
...,...,...,...,...
80611,610,116977,3.0,1479544278
80612,610,158238,5.0,1479545219
80613,610,160527,4.5,1479544998
80614,610,162350,3.5,1493849971


In [25]:
testsample = testRatings.limit(1)
testsample.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: integer (nullable = true)



In [26]:
predictions = model.transform(df4)
predictions.toPandas()

NameError: name 'df4' is not defined

In [ ]:
# Replace predicted NaN values with the average rating and evaluate the model:

avgRatings = df_data_ratings.select('rating').groupBy().avg().first()[0]
print ('The average rating in the dataset is: {}'.format(avgRatings))

evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')
print ('The root mean squared error for our model is: {}'.format(evaluator.evaluate(predictions.na.fill(avgRatings))))


In [ ]:
# Now exclude predicted NaN values and evaluate the model:

evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')
print ('The root mean squared error for our model is: {}'.format(evaluator.evaluate(predictions.na.drop())))

In [ ]:
def repeatALS(data, k=3, userCol='userId', itemCol='movieId', ratingCol='rating', metricName='rmse'):
    evaluations = []
    for i in range(0, k):  
        (trainingSet, testingSet) = data.randomSplit([k - 1.0, 1.0])
        als = ALS(userCol=userCol, itemCol=itemCol, ratingCol=ratingCol)
        model = als.fit(trainingSet)
        predictions = model.transform(testingSet)
        evaluator = RegressionEvaluator(metricName=metricName, labelCol='rating', predictionCol='prediction')
        evaluation = evaluator.evaluate(predictions.na.drop())
        print('Loop {}: {} = {}'.format(i + 1, metricName, evaluation))
        evaluations.append(evaluation)
    return sum(evaluations) / float(len(evaluations))


In [ ]:
print('RMSE = {}'.format(repeatALS(df_data_ratings, k=4)))
    

In [ ]:
def kfoldALS(data, k=3, userCol='userId', itemCol='movieId', ratingCol='rating', metricName='rmse'):
    evaluations = []
    weights = [1.0] * k
    splits = data.randomSplit(weights)
    for i in range(0, k):  
        testingSet = splits[i]
        trainingSet = spark.createDataFrame(sc.emptyRDD(), data.schema)
        for j in range(0, k):
            if i == j:
                continue
            else:
                trainingSet = trainingSet.union(splits[j])
        als = ALS(userCol=userCol, itemCol=itemCol, ratingCol=ratingCol)
        model = als.fit(trainingSet)
        predictions = model.transform(testingSet)
        evaluator = RegressionEvaluator(metricName=metricName, labelCol='rating', predictionCol='prediction')
        evaluation = evaluator.evaluate(predictions.na.drop())
        print('Loop {}: {} = {}'.format(i + 1, metricName, evaluation))
        evaluations.append(evaluation)
    return sum(evaluations) / float(len(evaluations))


In [ ]:
#print('RMSE = {}'.format(kfoldALS(df_data_ratings, k=4)))


In [ ]:
# #The Spark CrossValidator function performs a grid search as well as k-fold cross validation.

# from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# (trainingRatings, validationRatings) = df_data_ratings.randomSplit([80.0, 20.0])
# evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

# # rank = the number of latent factors in the model
# # maxIter = the maximum number of iterations
# # regParam = the regularization parameter

# paramGrid = ParamGridBuilder().addGrid(als.rank, [1, 5, 10]).addGrid(als.maxIter, [20]).addGrid(als.regParam, [0.05, 0.1, 0.5]).build()

# crossval = CrossValidator(estimator=als, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=10)
# cvModel = crossval.fit(trainingRatings)
# predictions = cvModel.transform(validationRatings)

# print('The root mean squared error for our model is: {}'.format(evaluator.evaluate(predictions.na.drop())))



In [ ]:
from pyspark.sql.functions import lit

def recommendMovies(model, user, nbRecommendations):
    # Create a Spark DataFrame with the specified user and all the movies listed in the ratings DataFrame
    dataSet = appendedRatings.select('movieId').distinct().withColumn('userId', lit(user))

    # Create a Spark DataFrame with the movies that have already been rated by this user
    moviesAlreadyRated =  appendedRatings.filter(appendedRatings.userId == user).select('movieId', 'userId')

    # Apply the recommender system to the data set without the already rated movies to predict ratings
    predictions = model.transform(dataSet.subtract(moviesAlreadyRated)).dropna().orderBy('prediction', ascending=False).limit(nbRecommendations).select('movieId', 'prediction')

    # Join with the movies DataFrame to get the movies titles and genres
    recommendations = predictions.join( df_data_movies, predictions.movieId ==  df_data_movies.movieId).select(predictions.movieId,  df_data_movies.title,  df_data_movies.genres, predictions.prediction)

#     recommendations.show(truncate=False)
    return dataSet, moviesAlreadyRated, predictions, recommendations


In [ ]:
appendedRatings = df_data_ratings

print('Recommendations for user 133:')
recommendMovies(model, 133, 10).toPandas()


In [ ]:
newUserID = int(ratingsPandas[['userId']].max()) +1
moviesPandas.sample(30)


In [ ]:
number_array = list()
number = 10
print ('Enter numbers in array: ')
for i in range(number):
    n = input("MovieID :")
    number_array.append(int(n))
print ('Your Selected Movie IDs: ',number_array)



In [ ]:
newUserID

In [ ]:
columns = ['userId','movieId','rating','timestamp']

appendedRatings = df_data_ratings
from pyspark.sql.types import IntegerType
import random
for i in range(number):
    newRow = spark.createDataFrame([(newUserID,number_array[i],float(random.randint(1,5)),int(0))], schema=columns)
    df2 = newRow.withColumn("userId", newRow["userId"].cast(IntegerType()))
    df3 = df2.withColumn("movieId", newRow["movieId"].cast(IntegerType()))
    df4 = df3.withColumn("timestamp", newRow["timestamp"].cast(IntegerType()))

    appendedRatings = appendedRatings.union(df4)
    

    
appendedRatings.toPandas()

In [ ]:
appendedRatings.printSchema()

In [ ]:
a,b,c,d = recommendMovies(model, 610, 5)

In [ ]:
a.toPandas()

In [ ]:
b.select('movieId').distinct().toPandas()

In [ ]:
c.toPandas()

In [ ]:
d.toPandas()

In [ ]:
w,x,y,z = recommendMovies(model, 611, 5)

In [ ]:
w.toPandas()

In [ ]:
x.toPandas()

In [ ]:
y.toPandas()

In [ ]:
df_data_ratings.printSchema()